# TensorFlow Tutorial
## A really boring work-through of the tutorial
David Norrish, 2/10/2017  
  
Available at: https://www.tensorflow.org/get_started/get_started  
Assumes you already have TensorFlow and (obviously) Jupyter Notebook installed  

Firstly, import the package to give Python access to all of TensorFlow's classes, methods, and symbols

In [1]:
import tensorflow as tf

A **tensor** is a set of primitive values shaped into an array of any number of dimensions. A tensor's rank is its number of dimensions. E.g.

- 3 
  - a rank 0 tensor; this is a scalar with shape []
- [1., 2., 3.] 
  - a rank 1 tensor; this is a vector with shape [3]
- [[1., 2., 3.], [4., 5., 6.]] 
  - a rank 2 tensor; a matrix with shape [2, 3]
- [[[1., 2., 3.]], [[7., 8., 9.]]]
  - a rank 3 tensor with shape [2, 1, 3]


A 'computational graph' is a series of TensorFlow operations arranged into a graph of nodes. Each node takes one or more tensor as inputs and produces a tensor as an output. A node can be a constant, which takes no inputs and stores its output value internally.  

TensorFlow's Core programs essentially consist of two discrete sections:

- Building the computational graph
- Running the computational graph  

Let's build a simple computational graph.

Let's create 2 floating point tensors to start with

In [2]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

(<tf.Tensor 'Const:0' shape=() dtype=float32>, <tf.Tensor 'Const_1:0' shape=() dtype=float32>)


Note that these constants are nodes that, when evaluated, would product 3.0 and 4.0.  
To evaluate the nodes, must run the computational graph within a **session**. A session encapsulates the control and state of the TensorFlow runtime.

In [3]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


Can build more complicated computations by combining Tensor nodes with operations (Operations are also nodes)

In [4]:
node3 = tf.add(node1, node2)
print("node3:", node3)
print("sess.run(node3):", sess.run(node3))

('node3:', <tf.Tensor 'Add:0' shape=() dtype=float32>)
('sess.run(node3):', 7.0)


TensorFlow provides a utility called 'TensorBoard' that can display a picture of the computational graph. We only have constants at the moment, which is boring. A graph can be parameterized to accept external inputs, known as placeholders. A placeholder is a promise to provide a value later. Let's use these for now.

In [5]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)


In [6]:
print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))

7.5
[ 3.  7.]


We can make this more complex by adding another operation

In [8]:
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b: 4.5}))

22.5


Variables allow us to add trainable parameters to a graph

In [9]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

To initialize all the variables in a TensorFlow program, you must explicitly call a special operation as follows:

In [10]:
init = tf.global_variables_initializer()
sess.run(init)